In [ ]:
# リスト 4.3.3〜4.3.5（コメント付与版）
# 目的：
# - IBM Watson NLU を Knowledge Studio で学習・デプロイしたカスタムモデル（温泉の泉質語彙など）で
#   エンティティ抽出を行う最小構成のサンプル。
# - ここで参照する学習データの例は、提示の CSV（"lemma","poscode","surface"... の同義語・表記ゆれ辞書）
#   と、「国民保養温泉地計画書」等の長文テキスト（泉質・効能の記述）である。
#   → これらを用いて Knowledge Studio 上でアノテーション（エンティティ型の付与）と学習を行い、
#      デプロイ済みモデルID（下記 EntitiesOptions(model=...)）を NLU から指定して利用する。

# --- 資格情報の設定 ---
# NLU の API キーとエンドポイント URL。
# 実運用では .env や Secret Manager 等から安全に読み込むのが望ましいが、
# ここでは学習コードの理解を優先し、置換前提のプレースホルダを残す。
nlu_credentials = {
    "apikey": "xxxx",  # ← Knowledge Studio と同一リージョンの NLU インスタンスの API キーに置換
    "url": "xxxx",  # ← 例: "https://api.jp-tok.natural-language-understanding.watson.cloud.ibm.com"
}

# --- 依存ライブラリの import ---
# json: API レスポンスの可読化表示に使用。
# IBM Watson NLU SDK: NaturalLanguageUnderstandingV1 本体と機能オプション（EntitiesOptions 等）。
# IAMAuthenticator: API キーでの認証に使用。
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import *
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# --- NLU クライアントの生成 ---
# version は API のスキーマバージョン。SDK の互換表に従って固定値を与える。
# authenticator は API キーをラップした IAM 認証器。
authenticator = IAMAuthenticator(nlu_credentials["apikey"])
nlu = NaturalLanguageUnderstandingV1(
    version="2019-07-12",  # SDK サンプル既定。プロジェクトの固定値として管理する。
    authenticator=authenticator,
)
# サービスのベース URL を設定（地域により異なる）。誤ると 404/401 になるので注意。
nlu.set_service_url(nlu_credentials["url"])

In [ ]:
# --------------------------------------------------------------------
# リスト 4.3.4 NLU 呼び出し用の共通関数
# --------------------------------------------------------------------
# 役割：
# - 任意テキストに対し、指定した Features（例: EntitiesOptions を含む Features）で解析を実行。
# - 返り値 JSON から、関心のあるトップレベルキー（"entities" など）を抽出して返す薄いラッパ。
#
# 引数:
# - text     : 解析対象テキスト（str）。UTF-8 で渡す。
# - features : ibm_watson.natural_language_understanding_v1.Features のインスタンス。
# - key      : 結果 JSON のトップレベルキー名（例 "entities", "relations"...）
#
# 返値:
# - response[key] : 指定キー配下の構造（通常は list）。KeyError を避けたい場合は防御的に .get を使う運用も可。
#
# 注意:
# - Language は NLU が自動判定するが、日本語短文では誤判定が起きうる。必要に応じ analyze(language="ja") を検討。
# - API のスロットリング（429）や一時障害（5xx）に備えたリトライ戦略は実運用で追加する。
def call_nlu(text, features, key):
    response = nlu.analyze(text=text, features=features).get_result()
    return response[key]

In [ ]:
# --------------------------------------------------------------------
# リスト 4.3.5 エンティティ抽出機能の呼び出し
# --------------------------------------------------------------------
# 入力文：
# - 温泉名（玉造温泉）と泉質（「カルシウム・ナトリウム－硫酸塩・塩化物泉」）が含まれる。
# - Knowledge Studio の学習データ（提示 CSV の「surface」群）では、泉質の多様な表記ゆれ
#   （例: Na-Cl泉 / ナトリウム－塩化物泉 / 塩化物泉 等）を "lemma"（正規化表層）にアライン可能。
# - 学習済みモデルは、これらの表記ゆれを同一エンティティ型（例: ThermalWaterType など、プロジェクト定義次第）
#   として抽出・正規化できることが期待される。
text = "大勢の観光客が温泉街を歩く島根県・玉造温泉(カルシウム・ナトリウム－硫酸塩・塩化物泉)は、環境大臣賞受賞。"

# Features 構築：
# - EntitiesOptions(model=...) に Knowledge Studio で「学習→評価→デプロイ」まで行ったカスタム NLU モデルの ID を指定。
# - これにより汎用辞書では捕捉しにくい「泉質」「効能」「温泉地名」などドメイン固有エンティティを高精度に抽出。
# - model ID はデプロイ単位で発行され、再学習・再デプロイで ID が変わる点に注意（構成管理で追跡推奨）。
features = Features(
    entities=EntitiesOptions(
        model="3c256ea1-d24c-433d-8fe2-695b5b4e658a"  # ← デプロイ済みカスタムモデル ID（例）。自身の環境の値に置換。
        # mentions=True などのオプションを使うと、同一エンティティの複数出現（表層）も取得できる。
    )
)

# API 呼び出し：
# - Knowledge Studio 由来のスキーマで、エンティティの "type"（定義した型名）や "text"（表層形）、
#   "disambiguation"（同義語の正規化結果）等が返る想定。
# - 解析の正否は、学習データのアノテーション品質（境界・ラベル一貫性）と表記ゆれ網羅性に依存。
ret = call_nlu(text, features, "entities")

# 結果表示：
# - 実務ではこの JSON から (lemma, type, offset, normalized_form) など必要項目を射影し、
#   ES などの検索インデックスへ保存・集計に回す。
# - ここでは学習確認のため、人間可読な整形表示を行う。
print(json.dumps(ret, indent=2, ensure_ascii=False))